In [1]:
!git pull

Already up to date.


In [1]:
# We need to decompress the data and store the in chunks optimal for the dataloader to read

from utils import unpack_h5
unpack_h5("/shared/usiddiqui/IJEPA/data_compressed/qg_medium_train.h5", "/shared/usiddiqui/IJEPA/data_uncompressed/qg_medium_train.h5")
unpack_h5("/shared/usiddiqui/IJEPA/data_compressed/qg_medium_test.h5", "/shared/usiddiqui/IJEPA/data_uncompressed/qq_medium_test.h5")
unpack_h5("/shared/usiddiqui/IJEPA/data_compressed/qg_medium_finetune.h5", "/shared/usiddiqui/IJEPA/data_uncompressed/qg_medium_finetune.h5")

Target file /shared/usiddiqui/IJEPA/data_uncompressed/qg_medium_train.h5 already exists. Skipping.
Target file /shared/usiddiqui/IJEPA/data_uncompressed/qq_medium_test.h5 already exists. Skipping.
Target file /shared/usiddiqui/IJEPA/data_uncompressed/qg_medium_finetune.h5 already exists. Skipping.


In [2]:
!python -u train_pretrain.py

2026-02-04 18:25:50,383 | INFO | Config saved to /shared/usiddiqui/IJEPA/checkpoints/vit_small_ijepa_20260204_182550
2026-02-04 18:25:50,423 | INFO | --- Dataset Loaded (Size: 100000) ---
Epoch 1:   0%|                                       | 0/391 [00:18<?, ?batch/s]
Traceback (most recent call last):
  File "/shared/usiddiqui/IJEPA/QG_Ijepa/train_pretrain.py", line 85, in <module>
    for images, labels, ctx_idx, trg_idx in pbar:
                                            ^^^^
  File "/opt/conda/lib/python3.13/site-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
               ^^^^^^^^
  File "/opt/conda/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 741, in __next__
    data = self._next_data()
  File "/opt/conda/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1524, in _next_data
    idx, data = self._get_data()
                ~~~~~~~~~~~~~~^^
  File "/opt/conda/lib/python3.13/site-packages/torch/utils/data/dataloader.p

Time for finetuning and evaluation.

In [10]:
#!python train_finetune.py

--- Training MLP Head | Results path: /content/drive/MyDrive/QG_IJEPA/finetune/vit_small_ijepa_20260203_040412 ---
Epoch 1: 100% 313/313 [01:32<00:00,  3.37it/s, acc=0.612, loss=0.644]
Epoch 2: 100% 313/313 [01:34<00:00,  3.32it/s, acc=0.638, loss=0.592]
Epoch 3: 100% 313/313 [01:34<00:00,  3.30it/s, acc=0.648, loss=0.636]
Epoch 4: 100% 313/313 [01:34<00:00,  3.30it/s, acc=0.65, loss=0.557]
--- Computing Final ROC AUC ---
100% 313/313 [01:34<00:00,  3.30it/s]
Figure(800x600)

Final AUC: 0.7074
Results saved to: /content/drive/MyDrive/QG_IJEPA/finetune/vit_small_ijepa_20260203_040412
